In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
filename = "dados/sharpe.xlsx"

### Risk Free

In [3]:
df_rf = pd.read_excel(filename, sheet_name="risk free")

In [4]:
df_rf_mensal = pd.DataFrame({'rf_media' : df_rf.groupby( [ "year", "month"] )['Risk_free'].mean()}).reset_index()

In [5]:
df_rf_mensal['rf_mensal'] = (1+df_rf_mensal['rf_media'])**22-1

In [6]:
df_rf_2010 = df_rf_mensal[df_rf_mensal['year'] >= 2010]['rf_mensal']

In [7]:
df = pd.read_excel(filename)

In [8]:
df.reset_index(drop=True, inplace=True)
df_rf_2010.reset_index(drop=True, inplace=True)

In [9]:
df = pd.concat( [df, df_rf_2010], axis=1)

In [10]:
df.set_index('Data', inplace=True)

## Retornos

In [11]:
df_ipca = df.loc[:, 'IPCA a.a.':'IPCA a.m.']

In [12]:
cols = [x for x in df.columns if "Adj" in x]

In [13]:
df_quotes = df[cols]

In [14]:
df_r = np.log(df_quotes) - np.log(df_quotes.shift(1))

In [15]:
# Calculando Retorno Real
df_rr = (df_r + 1).div((df_ipca['IPCA a.m.'].shift(1) + 1), axis=0) - 1

In [16]:
df_rr['rf'] = df['rf_mensal']

In [17]:
df_rr = df_rr[1:]

In [18]:
for col in cols:
    col_name_re = col.split(".")[0] + "_12m_ER"
    col_name_stdev = col.split(".")[0] + "_12m_STDEV"
    col_name_sharpe = col.split(".")[0] + "_12m_SHARPE"
    df_rr[col_name_re] = df_rr[col].rolling(window=12).mean()
    df_rr[col_name_stdev] = df_rr[col].rolling(window=12).std()
    df_rr[col_name_sharpe] = (df_rr[col_name_re] - df_rr['rf'])/df_rr[col_name_stdev]

In [19]:
df_valid = df_rr[12:]

In [20]:
df_valid.head()

,VALE3.SA.Adjusted,ITUB4.SA.Adjusted,B3SA3.SA.Adjusted,PETR4.SA.Adjusted,BBDC4.SA.Adjusted,PETR3.SA.Adjusted,ABEV3.SA.Adjusted,BBAS3.SA.Adjusted,MGLU3.SA.Adjusted,ITSA4.SA.Adjusted,...,GOLL4_12m_SHARPE,ECOR3_12m_ER,ECOR3_12m_STDEV,ECOR3_12m_SHARPE,HGTX3_12m_ER,HGTX3_12m_STDEV,HGTX3_12m_SHARPE,BVSP_12m_ER,BVSP_12m_STDEV,BVSP_12m_SHARPE
Data,,,,,,,,,,,,,,,,,,,,,
2011-02-01,-0.013203,0.034493,NaN,0.048447,-0.036667,0.076219,-0.018560,-0.005171,NaN,0.024278,...,-0.199075,NaN,NaN,NaN,0.068155,0.093946,0.624787,-0.003080,0.053071,-0.236265
2011-03-01,-0.058411,0.045716,NaN,-0.012191,0.081014,-0.004241,0.030390,-0.000921,NaN,0.014629,...,-0.134929,NaN,NaN,NaN,0.077805,0.099900,0.681701,-0.006383,0.050474,-0.318688
2011-04-01,-0.034604,-0.051506,NaN,-0.107297,-0.037492,-0.125892,0.152096,-0.024114,NaN,-0.021653,...,-0.201140,0.027401,0.055515,0.316336,0.079270,0.100399,0.691542,-0.006057,0.050212,-0.316582
2011-05-01,-0.023393,-0.038936,NaN,-0.060762,-0.017666,-0.074442,-0.030132,-0.038032,NaN,-0.031836,...,-0.175133,0.030434,0.053915,0.380427,0.077153,0.100608,0.668233,-0.002357,0.046345,-0.264970
2011-06-01,-0.020433,0.012218,-0.09631,-0.020670,0.017375,-0.027012,0.039196,-0.001301,0.01015,0.009774,...,-0.280515,0.023758,0.056714,0.241395,0.065791,0.102163,0.545432,-0.002510,0.046477,-0.270624


## Plot

In [21]:
import plotly.express as px

In [22]:
cols_sharpe = [x for x in df_valid.columns if "SHARPE" in x]

In [23]:
df_sharpe = df_valid[cols_sharpe]

In [24]:
df_sharpe.rename(columns={'VALE3_12m_SHARPE' : 'VALE3',
                        'ITUB4_12m_SHARPE' : 'ITUB4',
                        'B3SA3_12m_SHARPE' : 'B3SA3',
                        'PETR4_12m_SHARPE' : 'PETR4',
                        'BBDC4_12m_SHARPE' : 'BBDC4',
                        'PETR3_12m_SHARPE' : 'PETR3',
                        'ABEV3_12m_SHARPE' : 'ABEV3',
                        'BBAS3_12m_SHARPE' : 'BBAS3',
                        'MGLU3_12m_SHARPE' : 'MGLU3',
                        'ITSA4_12m_SHARPE' : 'ITSA4',
                        'JBSS3_12m_SHARPE' : 'JBSS3',
                        'LREN3_12m_SHARPE' : 'LREN3',
                        'WEGE3_12m_SHARPE' : 'WEGE3',
                        'GNDI3_12m_SHARPE' : 'GNDI3',
                        'NTCO3_12m_SHARPE' : 'NTCO3',
                        'SUZB3_12m_SHARPE' : 'SUZB3',
                        'RAIL3_12m_SHARPE' : 'RAIL3',
                        'BBDC3_12m_SHARPE' : 'BBDC3',
                        'RENT3_12m_SHARPE' : 'RENT3',
                        'LAME4_12m_SHARPE' : 'LAME4',
                        'EQTL3_12m_SHARPE' : 'EQTL3',
                        'RADL3_12m_SHARPE' : 'RADL3',
                        'VIVT4_12m_SHARPE' : 'VIVT4',
                        'SBSP3_12m_SHARPE' : 'SBSP3',
                        'BTOW3_12m_SHARPE' : 'BTOW3',
                        'UGPA3_12m_SHARPE' : 'UGPA3',
                        'BBSE3_12m_SHARPE' : 'BBSE3',
                        'BRFS3_12m_SHARPE' : 'BRFS3',
                        'VVAR3_12m_SHARPE' : 'VVAR3',
                        'CCRO3_12m_SHARPE' : 'CCRO3',
                        'BRDT3_12m_SHARPE' : 'BRDT3',
                        'GGBR4_12m_SHARPE' : 'GGBR4',
                        'BPAC11_12m_SHARPE' : 'BPAC11',
                        'HYPE3_12m_SHARPE' : 'HYPE3',
                        'HAPV3_12m_SHARPE' : 'HAPV3',
                        'KLBN11_12m_SHARPE' : 'KLBN11',
                        'SULA11_12m_SHARPE' : 'SULA11',
                        'ENGI11_12m_SHARPE' : 'ENGI11',
                        'COGN3_12m_SHARPE' : 'COGN3',
                        'TIMP3_12m_SHARPE' : 'TIMP3',
                        'CMIG4_12m_SHARPE' : 'CMIG4',
                        'IRBR3_12m_SHARPE' : 'IRBR3',
                        'SANB11_12m_SHARPE' : 'SANB11',
                        'EGIE3_12m_SHARPE' : 'EGIE3',
                        'ELET3_12m_SHARPE' : 'ELET3',
                        'CSAN3_12m_SHARPE' : 'CSAN3',
                        'PCAR3_12m_SHARPE' : 'PCAR3',
                        'YDUQ3_12m_SHARPE' : 'YDUQ3',
                        'TOTS3_12m_SHARPE' : 'TOTS3',
                        'BRML3_12m_SHARPE' : 'BRML3',
                        'QUAL3_12m_SHARPE' : 'QUAL3',
                        'BRAP4_12m_SHARPE' : 'BRAP4',
                        'ELET6_12m_SHARPE' : 'ELET6',
                        'FLRY3_12m_SHARPE' : 'FLRY3',
                        'CRFB3_12m_SHARPE' : 'CRFB3',
                        'CSNA3_12m_SHARPE' : 'CSNA3',
                        'BRKM5_12m_SHARPE' : 'BRKM5',
                        'AZUL4_12m_SHARPE' : 'AZUL4',
                        'CYRE3_12m_SHARPE' : 'CYRE3',
                        'TAEE11_12m_SHARPE' : 'TAEE11',
                        'EMBR3_12m_SHARPE' : 'EMBR3',
                        'CIEL3_12m_SHARPE' : 'CIEL3',
                        'MULT3_12m_SHARPE' : 'MULT3',
                        'MRFG3_12m_SHARPE' : 'MRFG3',
                        'CPFE3_12m_SHARPE' : 'CPFE3',
                        'ENBR3_12m_SHARPE' : 'ENBR3',
                        'MRVE3_12m_SHARPE' : 'MRVE3',
                        'GOAU4_12m_SHARPE' : 'GOAU4',
                        'USIM5_12m_SHARPE' : 'USIM5',
                        'BEEF3_12m_SHARPE' : 'BEEF3',
                        'IGTA3_12m_SHARPE' : 'IGTA3',
                        'CVCB3_12m_SHARPE' : 'CVCB3',
                        'GOLL4_12m_SHARPE' : 'GOLL4',
                        'ECOR3_12m_SHARPE' : 'ECOR3',
                        'HGTX3_12m_SHARPE' : 'HGTX3',
                        'BVSP_12m_SHARPE' : 'BVSP'}, inplace=True)

In [30]:
fig_sharpe_mensal = px.line(df_sharpe,
             title="Índice Sharpe Mensal",
             labels={
                 "variable": "Ativo",  "value": "Índice Sharpe"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_sharpe_mensal.show()

In [27]:
df_sharpe.describe()

,VALE3,ITUB4,B3SA3,PETR4,BBDC4,PETR3,ABEV3,BBAS3,MGLU3,ITSA4,...,MRVE3,GOAU4,USIM5,BEEF3,IGTA3,CVCB3,GOLL4,ECOR3,HGTX3,BVSP
count,112.000000,112.000000,97.000000,112.000000,112.000000,112.000000,112.000000,112.000000,97.000000,112.000000,...,112.000000,112.000000,112.000000,112.000000,112.000000,66.000000,112.000000,110.000000,112.000000,112.000000
mean,-0.162724,-0.058098,0.068294,-0.125511,-0.026409,-0.127264,-0.018323,-0.073910,0.159512,-0.030370,...,-0.034953,-0.230100,-0.161159,-0.089093,-0.022759,0.150997,-0.077596,-0.098649,-0.078319,-0.159232
std,0.352865,0.185791,0.276661,0.258991,0.218923,0.247955,0.331959,0.274775,0.442119,0.205888,...,0.186612,0.385957,0.327973,0.316572,0.272138,0.464183,0.336664,0.355552,0.380342,0.308191
min,-0.982818,-0.657424,-0.650081,-0.686382,-0.654568,-0.627878,-0.568777,-1.042662,-0.844855,-0.650340,...,-0.519445,-1.220841,-1.051817,-0.704729,-0.586482,-0.559720,-1.069140,-0.956145,-1.077499,-1.139030
25%,-0.418658,-0.162870,-0.112742,-0.331452,-0.168952,-0.307656,-0.318659,-0.195010,-0.138290,-0.157726,...,-0.163377,-0.440633,-0.413120,-0.328888,-0.202346,-0.150758,-0.267095,-0.301209,-0.321795,-0.358892
50%,-0.204878,-0.037080,0.080887,-0.183186,0.010028,-0.183063,-0.054713,-0.053167,0.037256,-0.020049,...,-0.046791,-0.158933,-0.125400,-0.137372,-0.015368,0.046507,-0.044736,-0.056786,-0.069955,-0.182668
75%,0.050538,0.079080,0.214606,0.114684,0.130620,0.094161,0.303637,0.105417,0.380546,0.121960,...,0.061471,0.091345,0.026792,0.077635,0.166114,0.340656,0.156058,0.166149,0.202605,0.075503
max,0.683123,0.356322,0.878099,0.401097,0.444770,0.379972,0.706444,0.422956,1.510757,0.436921,...,0.431892,0.435434,0.577420,0.698369,0.587887,1.458387,0.538419,0.666342,0.691542,0.470787


In [28]:
df_max = df_sharpe[df_sharpe.columns[df_sharpe.mean(axis=0) > 0.25]]
df_min = df_sharpe[df_sharpe.columns[df_sharpe.mean(axis=0) < -0.15]]

In [32]:
fig_sharpe_mensal_top = px.line(df_max,
              title="Índice Sharpe Mensal (Top)",
              labels={
                 "variable": "Ativo",  "value": "Índice Sharpe"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_sharpe_mensal_top.show()

In [33]:
fig_sharpe_mensal_bot = px.line(df_min,
              title="Índice Sharpe Mensal (Bottom)",
              labels={
                 "variable": "Ativo",  "value": "Índice Sharpe"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_sharpe_mensal_bot.show()

## Carteira Sharpe

In [34]:
dict1 = df_sharpe.to_dict()

In [35]:
from collections import OrderedDict

In [36]:
cols_sharpe_mod = list(df_sharpe.columns)

In [37]:
cols_sharpe_mod.remove("BVSP")

In [38]:
def topSharpeKeys(row, sharpe2):
    dict_sharpe = {}
    for col in cols_sharpe_mod:
        if abs(row[col]) > 0: 
            dict_sharpe[col] = row[col]
    ordered = OrderedDict(sorted(dict_sharpe.items(), reverse=True, key=lambda t: t[1]))
    sharpe_list = list(ordered.keys())[:5]
    if(sharpe2):
        sharpe_list.extend(list(ordered.keys())[-5:])
    return(sharpe_list)

In [39]:
def topSharpeValues(row, sharpe2):
    dict_sharpe = {}
    for col in cols_sharpe_mod:
        if abs(row[col]) > 0: 
            dict_sharpe[col] = row[col]
    ordered = OrderedDict(sorted(dict_sharpe.items(), reverse=True, key=lambda t: t[1]))
    sharpe_list = list(ordered.values())[:5]
    if(sharpe2):
        sharpe_list.extend([x*-1 for x in list(ordered.values())[-5:]])
    return(sharpe_list)

In [40]:
df_sharpe['top_sharpe_keys'] = df_sharpe.apply(lambda row : topSharpeKeys(row, False), axis = 1)
df_sharpe['top_sharpe_vals'] = df_sharpe.apply(lambda row : topSharpeValues(row, False), axis = 1)
df_sharpe['top_sharpe_keys_10'] = df_sharpe.apply(lambda row : topSharpeKeys(row, True), axis = 1)
df_sharpe['top_sharpe_vals_10'] = df_sharpe.apply(lambda row : topSharpeValues(row, True), axis = 1)

In [41]:
df_valid['portfolio_keys'] = df_sharpe['top_sharpe_keys'].shift(1)
df_valid['portfolio_vals'] = df_sharpe['top_sharpe_vals'].shift(1)
df_valid['portfolio_keys_10'] = df_sharpe['top_sharpe_keys_10'].shift(1)
df_valid['portfolio_vals_10'] = df_sharpe['top_sharpe_vals_10'].shift(1)

In [42]:
def getPortfolioReturns(row, sharpe2):
    returns = []
    for ref in row['portfolio_keys'][:5]:
        r = ref + ".SA.Adjusted"
        returns.append(row[r])
    if(sharpe2):
        for ref2 in row['portfolio_keys_10'][5:]:
            r2 = ref2 + ".SA.Adjusted"
            returns.append(row[r2]*-1)
    return sum(returns) / len(returns) 

In [43]:
df_final = df_valid[1:]

In [44]:
df_final['Carteira Sharpe'] = df_final.apply(lambda row : getPortfolioReturns(row, False), axis = 1)
df_final['Carteira Sharpe 10'] = df_final.apply(lambda row : getPortfolioReturns(row, True), axis = 1)

In [45]:
df_final['Carteira Sharpe Acumulada'] = np.cumprod(1 + df_final['Carteira Sharpe'].values) - 1
df_final['Carteira Sharpe 10 Acumulada'] = np.cumprod(1 + df_final['Carteira Sharpe 10'].values) - 1

In [46]:
df_final['IBOV Acumulado'] = np.cumprod(1 + df_final['BVSP.Adjusted'].values) - 1

In [47]:
df_plot1 = df_final[['Carteira Sharpe', 'Carteira Sharpe Acumulada', 'BVSP.Adjusted', 'IBOV Acumulado', 
                     'Carteira Sharpe 10', 'Carteira Sharpe 10 Acumulada']].copy()

In [48]:
df_plot1.rename(columns={'BVSP.Adjusted':'IBOV'}, 
                 inplace=True)

In [49]:
df_plot1['Carteira Sharpe'] = round(df_plot1['Carteira Sharpe']*100, 4)
df_plot1['Carteira Sharpe Acumulada'] = round(df_plot1['Carteira Sharpe Acumulada']*100, 4)
df_plot1['IBOV'] = round(df_plot1['IBOV']*100, 4)
df_plot1['IBOV Acumulado'] = round(df_plot1['IBOV Acumulado']*100, 4)
df_plot1['Carteira Sharpe 10'] = round(df_plot1['Carteira Sharpe 10']*100, 4)
df_plot1['Carteira Sharpe 10 Acumulada'] = round(df_plot1['Carteira Sharpe 10 Acumulada']*100, 4)

In [50]:
fig = px.line(df_plot1[['Carteira Sharpe', 'IBOV']],
             title="Retorno Mensal Carteira Sharpe x IBOV",
             labels={
                 "variable": "Ativo",  "value": "Retorno Mensal (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [66]:
df_plot2 = df_plot1[['Carteira Sharpe Acumulada', 'IBOV Acumulado']].copy()

In [67]:
df_plot2.rename(columns={'Carteira Sharpe Acumulada':'Carteira Sharpe',
                        'IBOV Acumulado' : 'IBOV'}, 
                 inplace=True)

In [68]:
fig2 = px.line(df_plot2,
             title="Retorno Acumulado Carteira Sharpe x IBOV",
             labels={
                 "variable": "Ativo",  "value": "Retorno Acumulado (%)"
             })
fig2.show()

In [52]:
df_final[['Carteira Sharpe', 'BVSP.Adjusted']].describe()

,Carteira Sharpe,BVSP.Adjusted
count,111.000000,111.000000
mean,0.007143,-0.002353
std,0.123971,0.066739
min,-0.581988,-0.357419
25%,-0.031120,-0.038894
50%,0.013254,0.000058
75%,0.063930,0.036565
max,0.377498,0.147261


## Carteira Sharpe 2

In [60]:
fig3 = px.line(df_plot1[['Carteira Sharpe 10', 'IBOV']],
             title="Retorno Mensal Carteira Sharpe 10 x IBOV",
             labels={
                 "variable": "Ativo",  "value": "Retorno Mensal (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig3.show()

In [69]:
df_plot3 = df_plot1[['Carteira Sharpe Acumulada', 'Carteira Sharpe 10 Acumulada', 'IBOV Acumulado']].copy()

In [70]:
df_plot3.rename(columns={'Carteira Sharpe Acumulada':'Carteira Sharpe',
                         'Carteira Sharpe 10 Acumulada' : 'Carteira Sharpe 10',
                        'IBOV Acumulado' : 'IBOV'}, 
                 inplace=True)

In [71]:
fig4 = px.line(df_plot3,
             title="Retorno Acumulado Carteira Sharpe 10 x IBOV",
             labels={
                 "variable": "Ativo",  "value": "Retorno Acumulado (%)"
             })
fig4.show()

In [55]:
df_final[['Carteira Sharpe', 'Carteira Sharpe 10', 'BVSP.Adjusted']].describe()

,Carteira Sharpe,Carteira Sharpe 10,BVSP.Adjusted
count,111.000000,111.000000,111.000000
mean,0.007143,0.006894,-0.002353
std,0.123971,0.065225,0.066739
min,-0.581988,-0.227346,-0.357419
25%,-0.031120,-0.018754,-0.038894
50%,0.013254,0.009924,0.000058
75%,0.063930,0.040905,0.036565
max,0.377498,0.231363,0.147261
